# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
import numpy as np
import glob
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping


We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_doc = np.random.permutation(open('/content/drive/MyDrive/upgrad/neural_data/project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/drive/MyDrive/upgrad/neural_data/project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

In [ ]:
# Check for possible image dimensions
dimensions = []
for i in range(len(train_doc)):
  path = os.path.join('/content/drive/MyDrive/upgrad/neural_data/project_data/train', train_doc[i].split(';')[0], '*')
  path = glob.glob(path)
  temp=[]
  img = imread(path[0])
  dimensions.append(img.shape)
# Extract unique values with set function
unique_dimensions = set(dimensions)
print("Total images : " +  str(len(dimensions)))
print("Image dimensions in training set : {}".format(unique_dimensions))

### Conclusion

We have 663 images and possible widths are : 120 & 360, whereas possible heights are : 160 & 360.

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
IMG_WIDTH = 120
IMG_HEIGHT = 120
CHANNELS = 3

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = list(range(0,30,2))#create a list of image numbers you want to use for a particular video
    print(len(img_idx))
    orig_batch_size = batch_size
    while True:
        #print('still running')
        t = np.random.permutation(folder_list)
        batch_size = orig_batch_size
        if(len(folder_list)%batch_size)==0:
            num_batches = int(len(folder_list)/batch_size)# calculate the number of batches
        else:
            num_batches = int((len(folder_list)//batch_size) + 1)# calculate the number of batches
        #print('len(folder_list)',len(folder_list))
        #print('num_batches calculated',num_batches)
        for batch in range(num_batches): # we iterate over the number of batches
            #print('starting batch',batch,'of ',num_batches)
            if(batch==num_batches-1) and (len(folder_list)%batch_size)!=0:
                batch_size =  int(len(folder_list)%batch_size)
                print('for final batch, size is ',batch_size)
            batch_data = np.zeros((batch_size,len(img_idx),IMG_WIDTH,IMG_HEIGHT,CHANNELS)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            #act_batch_data = np.zeros((batch_size,len(img_idx),100352))
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #print('img_idx',img_idx)
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    #print('read image')
                    image = resize(image, (IMG_WIDTH, IMG_HEIGHT))
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - np.mean(image[:,:,0]))/np.std(image[:,:,0])
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.mean(image[:,:,1]))/np.std(image[:,:,1])
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.mean(image[:,:,2]))/np.std(image[:,:,2])

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

                #act_batch_data[folder] = model.predict(batch_data[folder])
            #print('yielding batch now')
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/drive/MyDrive/upgrad/neural_data/project_data/train'
val_path = '/content/drive/MyDrive/upgrad/neural_data/project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

num_classes = 5
print('# classes =', num_classes)

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Input, Dropout
from keras.layers import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers, applications

#write your model here
STEPS = 15
inputs = Input(shape=(STEPS,IMG_WIDTH,IMG_HEIGHT,CHANNELS),name='input')
output_1 = TimeDistributed(keras.applications.vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(IMG_WIDTH,IMG_HEIGHT,CHANNELS)),name='Vgg16TimeDistribution')(inputs)
output_2 = TimeDistributed(Flatten())(output_1)
output_3 = GRU(500,return_sequences=False)(output_2)
predictions = Dense(5, activation='softmax')(output_3)

model = Model([inputs],predictions)
model.summary()













Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
for layer in model.layers:
    if layer.name == 'Vgg16TimeDistribution':
        layer.trainable = False

In [ ]:
optimiser = optimizers.Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.01)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
print('num_train_sequences',len(train_doc))
print('num_val_sequences',len(val_doc))
print('batch size',batch_size)
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
print('steps_per_epoch',steps_per_epoch)
print('validation_steps',validation_steps)

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, verbose=5,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1)

In [ ]:
plt.figure()
plt.title('Training vs Validation Loss')
plt.plot(range(0,10),model.history.history['val_loss'],label='validation_loss')
plt.plot(range(0,10),model.history.history['loss'],label='training_loss')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.title('Training vs Validation Accuracy')
plt.plot(range(0,10),model.history.history['val_categorical_accuracy'],label='validation_accuracy')
plt.plot(range(0,10),model.history.history['categorical_accuracy'],label='training_accuracy')
plt.legend()
plt.show()

In [ ]:
#Model Optimization
STEPS = 15
inputs = Input(shape=(STEPS,IMG_WIDTH,IMG_HEIGHT,CHANNELS),name='input')
output_1 = TimeDistributed(keras.applications.vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(IMG_WIDTH,IMG_HEIGHT,CHANNELS)),name='Vgg16TimeDistribution')(inputs)
output_2 = TimeDistributed(keras.layers.Dropout(0.9))(output_1)
output_3 = TimeDistributed(keras.layers.BatchNormalization())(output_2)
output_4 = TimeDistributed(Flatten())(output_3)
output_5 = GRU(300,return_sequences=False)(output_4)
predictions = Dense(5, activation='softmax')(output_5)

model_2 = Model([inputs],predictions)
model_2.summary()

In [ ]:
for layer in model_2.layers:
    if layer.name == 'Vgg16TimeDistribution':
        layer.trainable = False

In [ ]:
model_2.summary()

In [ ]:
optimiser = keras.optimizers.Adam()#write your optimizer
model_2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model_name = 'model2_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=5, save_best_only=True, save_weights_only=True, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2)

callbacks_list = [checkpoint, LR]

In [ ]:
history2 = model_2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, verbose=5,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None)

In [ ]:
plt.figure()
plt.title('Training vs Validation Loss')
plt.plot(range(0,10),model2.history.history['val_loss'],label='validation_loss')
plt.plot(range(0,10),model2.history.history['loss'],label='training_loss')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.title('Training vs Validation Accuracy')
plt.plot(range(0,10),model.history.history['val_categorical_accuracy'],label='validation_accuracy')
plt.plot(range(0,10),model.history.history['categorical_accuracy'],label='training_accuracy')
plt.legend()
plt.show()

In [ ]:
#Model Optimization
STEPS = 15
inputs = Input(shape=(STEPS,IMG_WIDTH,IMG_HEIGHT,CHANNELS),name='input')
#input_1 = keras.layers.Dropout(0.2)(inputs)
output_1 = TimeDistributed(keras.applications.vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(IMG_WIDTH,IMG_HEIGHT,CHANNELS)),name='Vgg16TimeDistribution')(inputs)
output_2 = TimeDistributed(keras.layers.Dropout(0.8))(output_1)
output_3 = TimeDistributed(keras.layers.BatchNormalization())(output_2)
output_4 = TimeDistributed(Flatten())(output_3)
output_5 = GRU(100,return_sequences=False)(output_4)
output_6 = keras.layers.Dropout(0.8)(output_5)
output_7 = keras.layers.BatchNormalization()(output_6)
predictions = Dense(5, activation='softmax')(output_7)

model_3 = Model([inputs],predictions)
model_3.summary()

In [ ]:
for layer in model_3.layers:
    if layer.name == 'Vgg16TimeDistribution':
        layer.trainable = False

In [ ]:
model_3.summary()


In [ ]:
optimiser = keras.optimizers.RMSprop()#write your optimizer
model_3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model_name = 'model3_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=5, save_best_only=True, save_weights_only=True, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2)

callbacks_list = [checkpoint, LR]

In [ ]:
model_3.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, verbose=5,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None)

In [ ]:
#Model Optimization using bidirectional GRU layer
STEPS = 15
inputs = Input(shape=(STEPS,IMG_WIDTH,IMG_HEIGHT,CHANNELS),name='input')
#input_1 = keras.layers.Dropout(0.2)(inputs)
output_1 = TimeDistributed(keras.applications.vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(IMG_WIDTH,IMG_HEIGHT,CHANNELS)),name='Vgg16TimeDistribution')(inputs)
output_2 = TimeDistributed(keras.layers.Dropout(0.8))(output_1)
output_3 = TimeDistributed(keras.layers.BatchNormalization())(output_2)
output_4 = TimeDistributed(Flatten())(output_3)
output_5 = keras.layers.Bidirectional(GRU(100,return_sequences=False))(output_4)
output_6 = keras.layers.Dropout(0.8)(output_5)
output_7 = keras.layers.BatchNormalization()(output_6)
predictions = Dense(5, activation='softmax')(output_7)

model_4 = Model([inputs],predictions)
model_4.summary()

In [ ]:
for layer in model_4.layers:
    if layer.name == 'Vgg16TimeDistribution':
        layer.trainable = False

In [ ]:
model_4.summary()

In [ ]:
optimiser = keras.optimizers.RMSprop() #write your optimizer
model_4.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model_name = 'model4_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=5, save_best_only=True, save_weights_only=True, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2)

callbacks_list = [checkpoint, LR]

In [ ]:
model_4.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, verbose=5,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None)

### Trying Conv3D Library to create model

In [ ]:
train_path = '/content/drive/MyDrive/upgrad/neural_data/project_data/train'
val_path = '/content/drive/MyDrive/upgrad/neural_data/project_data/val'
num_train_sequences = len(train_doc) #change training data size for initial phase
num_val_sequences = len(val_doc) #val training data can be kept as such
num_epochs = 30 # choose the number of epochs
batch_size = 100 #experiment with the batch size

### Generators

In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)

    #note using alternate 15 images here
    img_idx = list(range(0,30,2))#create a list of image numbers you want to use for a particular video
    #img_idx = list(range(5, 25)) #taking middle frames only
    orig_batch_size = batch_size
    while True:

        t = np.random.permutation(folder_list)
        batch_size = orig_batch_size
        if(len(folder_list)%batch_size)==0:
            num_batches = int(len(folder_list)/batch_size)# calculate the number of batches
        else:
            num_batches = int((len(folder_list)//batch_size) + 1)# calculate the number of batches

        for batch in range(num_batches): # we iterate over the number of batches

            if(batch==num_batches-1) and (len(folder_list)%batch_size)!=0:
                batch_size =  int(len(folder_list)%batch_size)
                #print('for final batch, size is ',batch_size)

            batch_data = np.zeros((batch_size,len(img_idx),IMG_WIDTH,IMG_HEIGHT,CHANNELS)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output

            #act_batch_data = np.zeros((batch_size,len(img_idx),100352))
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder

                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #print('img_idx',img_idx)
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    #print('read image')
                    image = resize(image, (IMG_WIDTH, IMG_HEIGHT))
                    #(image[:,:,0] - np.mean(image[:,:,0]))/np.std(image[:,:,0])
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - np.mean(image[:,:,0]))/np.std(image[:,:,0])
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.mean(image[:,:,1]))/np.std(image[:,:,1])
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.mean(image[:,:,2]))/np.std(image[:,:,2])
                    #(image[:,:,2] - np.percentile(image[:,:,2], 5))/(np.percentile(image[:,:,2], 95) - np.percentile(image[:,:,2], 5))

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

### Callbacks

In [ ]:
curr_dt_time = datetime.datetime.now()
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=True, mode='auto', period=1)

#pateince = after how many epochs the learning rate should be reduced.
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4 , cooldown=1, verbose=1)
callbacks_list = [LR, checkpoint]

#### Steps per Epoch, Validation steps

In [ ]:
print('num_train_sequences',len(train_doc))
print('num_val_sequences',len(val_doc))
print('batch size',batch_size)
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
print('steps_per_epoch',steps_per_epoch)
print('validation_steps',validation_steps)

In [ ]:
model_conv3d = Sequential()

model_conv3d.add(Conv3D(16, kernel_size=(3, 3, 3), activation = 'relu', input_shape = (15, 120, 120, 3), padding='same'))
model_conv3d.add(BatchNormalization())
model_conv3d.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_conv3d.add(Conv3D(32, kernel_size = (3, 3, 3), activation = 'relu', padding='same'))
model_conv3d.add(BatchNormalization())
model_conv3d.add(MaxPooling3D(pool_size=(1, 2, 2), strides = (1,2,2)))

model_conv3d.add(Conv3D(64, kernel_size = (3, 3, 3), activation = 'relu', padding='same'))
model_conv3d.add(BatchNormalization())
model_conv3d.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_conv3d.add(Conv3D(128, kernel_size = (3, 3, 3), activation = 'relu', padding='same'))
model_conv3d.add(BatchNormalization())
model_conv3d.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_conv3d.add(Flatten())
model_conv3d.add(Dropout(0.5))
model_conv3d.add(Dense(128, activation='relu'))
model_conv3d.add(Dropout(0.5))

model_conv3d.add(Dense(5, activation = 'softmax'))

In [ ]:
model_conv3d.summary()

In [ ]:
optimiser = optimizers.Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
# Fit the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)



### Summary

The CONV3D model achieved higher accuracy on the validation Set with 1 million parameters. This will be our final choice of model.